In [142]:
from profiles.models import *
from postings.models import *
from datetime import date
from django.db.models.fields import CharField
import re
from collections import Counter
from utils.glove import Glove

In [131]:
def collect_char_fields(Model):
    full_text = []
    char_fields = []
    for field in Model._meta.fields:
        if type(field) == CharField:
            char_fields.append(field.name)
    print("{} Fields: {}".format(Model, ' '.join(char_fields)))
    for p in Model.objects.all():
        for field in char_fields:
            full_text.append(getattr(p, field))
    return full_text

def split_to_words(full_text):
    out = []
    for t in full_text:
        out += re.split('\W+', t)
    return out

def filter_non_alpha(full_text):
    filtered_text = []
    for t in full_text:
        all_alpha = True
        for c in t:
            if not c.isalpha():
                all_alpha = False
                break

        if all_alpha and t:
            filtered_text.append(t)
    return filtered_text

def counter_filter(tokens, threshold):
    out = []
    c = Counter(tokens)
    for i in c:
        if c[i] >= threshold:
            out.append(i)
    return out
    

In [132]:
full_text = []
full_text += collect_char_fields(Education)
full_text += collect_char_fields(Experience)
full_text += collect_char_fields(Posting)

<class 'profiles.models.Education'> Fields: school_name degree field_of_study description
<class 'profiles.models.Experience'> Fields: company title description
<class 'postings.models.Posting'> Fields: id title company city state country source description search_title


In [133]:
tokenized_text = split_to_words(full_text)

In [134]:
filterd_tokens = filter_non_alpha(tokenized_text)

In [135]:
lowered_filterd_tokens = [w.lower() for w in filterd_tokens]

In [139]:
dictionary = counter_filter(lowered_filterd_tokens, 10)

In [149]:
glove_file = 'data/glove.6B.50d.txt'
out_file = 'data/filtered.glove.6B.50d.txt'

f = open(glove_file,'r')
w = open(out_file,'w')
for line in f:
    splitLine = line.split()
    word = splitLine[0]
    if word in dictionary:
        w.write(line)
    
f.close()
w.close()